Data Preprocessing
========================

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import argparse

In [2]:
df = pd.read_csv("revenue_preprocess.csv")
df.head()

,tic,datafqtr,cshoq,cshtrq,capxy,oiadpq,niq,xoprq,teqq,atq,...,MVA,CFROI,conm,revtq,SALE PRICE,Real GDP SA(billion),M2 SA(billion),M2-M1 SA(billion),M2V,Prime Rate(%)
0,NLP,2010Q1,10.666,263614.0,0.0,0.209,-2.568,11.450,63.433,309.571,...,-245.5828,NaN,NTS REALTY HOLDINGS LP,0.0,596771.895388,16582.710,8509.766667,6814.300000,1.735,3.25
1,NLP,2010Q2,10.666,235341.0,0.0,-0.307,-2.608,12.143,59.999,304.493,...,-257.3527,NaN,NTS REALTY HOLDINGS LP,0.0,623600.673062,16743.162,8597.600000,6884.033333,1.742,3.25
2,NLP,2010Q3,10.666,159175.0,0.0,0.336,-3.197,11.950,55.925,299.536,...,-254.1529,NaN,NTS REALTY HOLDINGS LP,0.0,645188.876686,16872.266,8682.300000,6936.000000,1.744,3.25
3,NLP,2010Q4,10.666,163338.0,0.0,0.642,-3.023,11.716,52.167,328.566,...,-228.8967,NaN,NTS REALTY HOLDINGS LP,0.0,956586.999871,16960.864,8793.633333,6978.466667,1.741,3.25
4,NLP,2011Q1,10.666,177414.0,0.0,0.476,-3.392,12.967,48.098,325.605,...,-281.8112,NaN,NTS REALTY HOLDINGS LP,0.0,607552.961452,16920.632,8907.066667,7036.100000,1.724,3.25


# Rename columns for better readability

In [3]:
# Rename DataFrame columns to follow camelCase convention and match original definitions from the images

# Basic company identifiers
df = df.rename(columns={
    'tic': 'ticker',  # Stock ticker symbol
    'datafqtr': 'fiscalQuarter',  # Fiscal quarter date
    'conm': 'companyName'  # Company name
})

# Target variables (what we're trying to predict)
df = df.rename(columns={
    'EBITDA': 'EBITDA',  # Earnings Before Interest, Taxes, Depreciation and Amortization (target2)
    'SALE PRICE': 'salePrice'  # Mean sale price per quarter (from additional variables)
})

# Basic financial variables (raw data from financial statements)
df = df.rename(columns={
    'cshoq': 'commonSharesOutstanding_',  # Number of common shares outstanding
    'cshtrq': 'commonSharesTraded_',  # Common shares traded
    'capxy': 'capitalExpenditure_',  # Funds used to acquire/upgrade physical assets
    'niq': 'netIncome_',  # Net income (profit after all expenses)
    'xoprq': 'operatingIncome_',  # Operating income after depreciation
    'teqq': 'shareholdersEquity_',  # Total shareholders' equity
    'atq': 'totalAssets_',  # Total assets on balance sheet
    'epsfiq': 'EPS_'  # Earnings per share (profit per outstanding share)
})

# Engineered financial features (calculated metrics)
df = df.rename(columns={
    'Debt_to_TA': 'debtToTotalAssets',  # Total debt divided by total assets
    'NI_to_Asset': 'niToAsset',  # Net income divided by average assets
    'ROA': 'ROA',  # Return on assets (net income/total assets)
    'ROE': 'ROE',  # Return on equity (net income/shareholders' equity)
    'Revenue_growth': 'revenueGrowth',  # Quarter-over-quarter revenue growth
    'NI_growth': 'netIncomeGrowth',  # Quarter-over-quarter net income growth
    'Quick_ratio': 'quickRatio',  # (Current assets - inventories)/current liabilities
    'RSI': 'rsi',  # Relative Strength Index (momentum indicator)
    'MVA': 'mva',  # Market Value Added (not explicitly defined in images)
    'CFROI': 'cfroi'  # Cash flow return on investment (OCF/Capital Employed)
})

# Macroeconomic variables (external factors)
df = df.rename(columns={
    'Real GDP SA(billion)': 'realGDPSA',  # Seasonally adjusted real GDP
    'M2 SA(billion)': 'm2SA',  # Seasonally adjusted M2 money supply
    'M2-M1 SA(billion)': 'm2MinusM1SA',  # Difference between M2 and M1 money supply
    'M2V': 'm2Velocity',  # Velocity of M2 money supply
    'Prime Rate(%)': 'primeRate'  # Benchmark interest rate
})

# Note: Some variables from the images weren't found in the original column list:
# - Revenue (revtq) - appears to be missing from original columns
# - Rooms Expenses (gmrmexpoq)
# - Room Revenue (gmrnewq)
# - Various homebuilding inventory metrics
# These would need to be added if present in the dataset


In [4]:
print(df.columns.tolist())

['ticker', 'fiscalQuarter', 'commonSharesOutstanding_', 'commonSharesTraded_', 'capitalExpenditure_', 'oiadpq', 'netIncome_', 'operatingIncome_', 'shareholdersEquity_', 'totalAssets_', 'EPS_', 'debtToTotalAssets', 'niToAsset', 'ROA', 'ROE', 'revenueGrowth', 'netIncomeGrowth', 'quickRatio', 'rsi', 'mva', 'cfroi', 'companyName', 'revtq', 'salePrice', 'realGDPSA', 'm2SA', 'm2MinusM1SA', 'm2Velocity', 'primeRate']


# Company selection

In [5]:
unique_tic_names = df['ticker'].unique()
print(unique_tic_names)
print(len(unique_tic_names))

['NLP' 'NMRK' 'ASPS' 'ATMFF.1' 'ALBT' 'AOXY' 'MAMP' 'MAYS' 'ARL' 'MMI'
 'MNGGF' 'MODVF' 'MRCBF' 'TH' 'TNL' 'PGCG' 'PKTEF' 'BCKMF' 'BEEP' 'BEKE'
 'MYHI.1' 'NEN' 'XIN' 'YYLRA' 'ZIPR' 'VTMX' 'VWTR' 'BSIRY' 'BSKS' 'CBRE'
 'MYCB' 'GZTGF' 'NNP.' 'NTPIF' 'PRLEQ' 'PRRE' 'PUB.Z' 'CDJM' 'TPHS' 'TPRP'
 'UK' 'GV' 'GYRO' 'CHIT' 'RMAX' 'ROII.1' 'SGD' 'RDFN' 'GEBRF' 'GGEI'
 'CIGI' 'HOFD' 'HOUS' 'CKX' 'CNR.1' 'TCN' 'CTCLY' 'CTYV' 'CWK' 'WAYS'
 'WEWKQ' 'WFICF' 'STRS' 'SVAUF' 'NXLCF' 'NYC' 'OMH' 'OPAD' 'OPEN' 'UOKA'
 'VAMA.1' 'VLTC' 'HMT.' 'EJ' 'EUDA' 'EXPI' 'SYSW' 'SYT' 'TCI' 'FCXXF'
 'FHRT' 'DGTC' 'DHOXY' 'DNK' 'DOUG' 'NWBA' 'VSE' 'MDPCF' 'MHIVF' '3CPTA'
 '5305B' '7634B' 'ACAN' 'GNLAF' 'LCANF' 'LEJUY' 'WLSE' 'WSC' 'WZI.H'
 'LRHC' 'LTFD' 'LUXH' 'CXW' 'IRS' 'JKA' 'JLL' 'OMAGQ' 'SIRC' 'SRG' 'SRRE'
 'STHO' 'STON' 'GBR' 'BPY' 'BREUF' 'FTHM' 'FRPH' 'FRWL' 'FSV' 'GADS'
 'RLLMF' 'MLLS' 'MLP' 'IRCP']
124


In [6]:
nyc_real_estate_tickers = [
    'NMRK',  # Newmark Group, Inc.
    'CBRE',  # CBRE Group, Inc.
    'CWK',   # Cushman & Wakefield plc
    'DOUG',  # Douglas Elliman Inc.
    'JLL',   # Jones Lang LaSalle Inc.
    'HOUS',  # Anywhere Real Estate Inc.
    'RMAX',  # RE/MAX Holdings, Inc.
    'EXPI',  # eXp World Holdings, Inc.
    'OPEN',  # Opendoor Technologies Inc.
    'OPAD',  # Offerpad Solutions Inc.
    'BEKE',  # KE Holdings Inc. (Limited NYC presence)
    'RDFN',  # Redfin Corporation
    'CIGI',  # Colliers International Group Inc.
    'FSV',   # FirstService Corporation
    'TCN',   # Tricon Residential Inc.

    # The following are private companies without tickers
    # 'Silverstein Properties',
    # 'Thor Equities',
    # 'JDS Development Group',
    # 'Charney Companies'
]
# Check if these tickers are in the DataFrame, which are not in the original list
for ticker in nyc_real_estate_tickers:
    if ticker not in df['ticker'].values:
        print(f"{ticker} is not in the DataFrame")
    else:
        print(f"{ticker} is in the DataFrame")

NMRK is in the DataFrame
CBRE is in the DataFrame
CWK is in the DataFrame
DOUG is in the DataFrame
JLL is in the DataFrame
HOUS is in the DataFrame
RMAX is in the DataFrame
EXPI is in the DataFrame
OPEN is in the DataFrame
OPAD is in the DataFrame
BEKE is in the DataFrame
RDFN is in the DataFrame
CIGI is in the DataFrame
FSV is in the DataFrame
TCN is in the DataFrame


In [ ]:
revenue_df = df[df['ticker'].isin(nyc_real_estate_tickers)]
revenue_df.head()

,ticker,fiscalQuarter,commonSharesOutstanding_,commonSharesTraded_,capitalExpenditure_,oiadpq,netIncome_,operatingIncome_,shareholdersEquity_,totalAssets_,...,mva,cfroi,companyName,revtq,salePrice,realGDPSA,m2SA,m2MinusM1SA,m2Velocity,primeRate
16,NMRK,2015Q4,NaN,NaN,12.133,-8.982,-2.803,1142.439,804.034,1657.930,...,NaN,0.415438,NEWMARK GROUP INC,0.0,1.281597e+06,18892.206,12290.766667,9224.500000,1.500,3.290323
17,NMRK,2016Q1,NaN,NaN,12.133,-8.982,-2.803,1142.439,804.034,1657.930,...,NaN,0.415438,NEWMARK GROUP INC,0.0,1.177307e+06,19001.690,12571.066667,9435.233333,1.474,3.500000
18,NMRK,2016Q2,NaN,NaN,12.133,-8.982,-2.803,1142.439,804.034,1657.930,...,NaN,0.415438,NEWMARK GROUP INC,0.0,1.336356e+06,19062.709,12772.800000,9546.333333,1.465,3.500000
19,NMRK,2016Q3,NaN,NaN,12.133,-8.982,-2.803,1142.439,804.034,1657.930,...,NaN,0.415438,NEWMARK GROUP INC,0.0,9.994895e+05,19197.938,12965.933333,9675.100000,1.457,3.500000
20,NMRK,2016Q4,NaN,NaN,27.260,153.008,168.401,1123.520,983.783,2534.688,...,NaN,-0.574873,NEWMARK GROUP INC,0.0,1.111994e+06,19304.352,13171.533333,9830.033333,1.449,3.545082


In [ ]:
print(revenue_df.columns.tolist())

['ticker', 'fiscalQuarter', 'commonSharesOutstanding_', 'commonSharesTraded_', 'capitalExpenditure_', 'oiadpq', 'netIncome_', 'operatingIncome_', 'shareholdersEquity_', 'totalAssets_', 'EPS_', 'debtToTotalAssets', 'niToAsset', 'ROA', 'ROE', 'revenueGrowth', 'netIncomeGrowth', 'quickRatio', 'rsi', 'mva', 'cfroi', 'companyName', 'revtq', 'salePrice', 'realGDPSA', 'm2SA', 'm2MinusM1SA', 'm2Velocity', 'primeRate']


In [ ]:
print(revenue_df.head(3))

   ticker fiscalQuarter  commonSharesOutstanding_  commonSharesTraded_  \
16   NMRK        2015Q4                       NaN                  NaN   
17   NMRK        2016Q1                       NaN                  NaN   
18   NMRK        2016Q2                       NaN                  NaN   

    capitalExpenditure_  oiadpq  netIncome_  operatingIncome_  \
16               12.133  -8.982      -2.803          1142.439   
17               12.133  -8.982      -2.803          1142.439   
18               12.133  -8.982      -2.803          1142.439   

    shareholdersEquity_  totalAssets_  ...  mva     cfroi        companyName  \
16              804.034       1657.93  ...  NaN  0.415438  NEWMARK GROUP INC   
17              804.034       1657.93  ...  NaN  0.415438  NEWMARK GROUP INC   
18              804.034       1657.93  ...  NaN  0.415438  NEWMARK GROUP INC   

    revtq     salePrice  realGDPSA          m2SA  m2MinusM1SA  m2Velocity  \
16    0.0  1.281597e+06  18892.206  12290.76

In [ ]:
print(revenue_df.head(3))

   ticker fiscalQuarter  commonSharesOutstanding_  commonSharesTraded_  \
16   NMRK        2015Q4                       NaN                  NaN   
17   NMRK        2016Q1                       NaN                  NaN   
18   NMRK        2016Q2                       NaN                  NaN   

    capitalExpenditure_  oiadpq  netIncome_  operatingIncome_  \
16               12.133  -8.982      -2.803          1142.439   
17               12.133  -8.982      -2.803          1142.439   
18               12.133  -8.982      -2.803          1142.439   

    shareholdersEquity_  totalAssets_  ...  mva     cfroi        companyName  \
16              804.034       1657.93  ...  NaN  0.415438  NEWMARK GROUP INC   
17              804.034       1657.93  ...  NaN  0.415438  NEWMARK GROUP INC   
18              804.034       1657.93  ...  NaN  0.415438  NEWMARK GROUP INC   

    revtq     salePrice  realGDPSA          m2SA  m2MinusM1SA  m2Velocity  \
16    0.0  1.281597e+06  18892.206  12290.76

In [ ]:
revenue_df.to_csv("revenue_new.csv", index=False)
print("New CSV file created: revenue_new.csv")

New CSV file created: revenue_new.csv


In [ ]:
revenue_df